## Framework imports

In [ ]:
from noronha.tools.publish import Publisher
from noronha.tools.shortcuts import data_path, tmp_path, require_dataset

## Application imports

In [ ]:
from joblib import dump, load
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_predict
from sklearn import metrics as sk_metrics

## Training parameters

In [ ]:
alpha = 0.001

## Load dataset

In [ ]:
ds_path = data_path(model='sentiment-clf', file_name='tweets.csv')

In [ ]:
dataset = pd.read_csv(ds_path, encoding='utf-8')
dataset.count()

In [ ]:
dataset.head()

In [ ]:
tweets = dataset["Text"].values
tweets

In [ ]:
classes = dataset["Classificacao"].values
classes

## Validation sample

In [ ]:
testes = ["Esse governo está no início, vamos ver o que vai dar",
          "Estou muito feliz com o governo de São Paulo esse ano",
          "O estado de Minas Gerais decretou calamidade financeira!!!",
          "A segurança desse país está deixando a desejar",
          "O governador de Minas é do PT",
          "O prefeito de São Paulo está fazendo um ótimo trabalho"]

## Train model

In [ ]:
vectorizer = CountVectorizer(ngram_range = (1, 2))
freq_tweets = vectorizer.fit_transform(tweets)

modelo = MultinomialNB(alpha=alpha)
modelo.fit(freq_tweets, classes)

## Validate model

In [ ]:
resultados = cross_val_predict(modelo, freq_tweets, classes, cv = 10)
resultados

In [ ]:
metrics = {'accuracy': sk_metrics.accuracy_score(classes, resultados)}
metrics

In [ ]:
print(sk_metrics.classification_report(classes, resultados))

In [ ]:
print(pd.crosstab(classes, resultados, rownames = ["Real"], colnames = ["Predito"], margins = True))

## Model publish

In [ ]:
pipe = Pipeline([("vectorizer", vectorizer), ("model", modelo)])
dump(pipe, tmp_path("sentiments.joblib"))

In [ ]:
Publisher()(
    src_path=tmp_path(),
    model_name='sentiment-clf',
    details=dict(
        params={'alpha': alpha},
        metrics=metrics,
    )
)